In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import numpy as np
from PIL import Image
import pathlib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import EfficientNetB7, preprocess_input
from tensorflow.keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading In Data

In [ ]:
data_dir = "/content/drive/MyDrive/lung_image_sets"
data = ImageDataGenerator(validation_split=0.2)
BATCH_SIZE = 32

X=224
Y=224

training = data.flow_from_directory(data_dir,
                                    class_mode = 'categorical',
                                    target_size = (X, Y),
                                    color_mode = "rgb",
                                    batch_size = BATCH_SIZE,
                                    shuffle = False,
                                    subset = "training",
                                    seed = 89)

validation = data.flow_from_directory(data_dir,
                                      class_mode = 'categorical',
                                      target_size = (X, Y),
                                      color_mode = "rgb",
                                      batch_size = BATCH_SIZE,
                                      shuffle = False,
                                      subset = 'validation',
                                      seed = 89)

Found 12008 images belonging to 3 classes.
Found 3002 images belonging to 3 classes.


# Displaying Sample Data

# Loading Model

In [ ]:
base_model = EfficientNetB7(
    input_shape=(X, Y, 3),
    weights = 'imagenet',
    include_top = False)

x = GlobalAveragePooling2D()(base_model.output)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation = 'relu')(x)
y = Dense(3, activation = 'softmax')(x)

model = Model(inputs=base_model.input, outputs=y)

258076736/258076736 [==============================] - 1s 0us/step


In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
model.fit(training,
          validation_data=validation,
          epochs=50)

Epoch 1/50


KeyboardInterrupt: ignored

The Model above struggled with overfitting. The validation accuracy was 33% while the training accuracy was 95.
The next model will introduce regularization to combat overfitting.

Model v2

In [ ]:
base_model = EfficientNetB7(
    input_shape=(X, Y, 3),
    weights = 'imagenet',
    include_top = False)

x = GlobalAveragePooling2D()(base_model.output)
x = Flatten()(x)
x = Dense(128, activation='relu', activity_regularizer=tf.keras.regularizers.L2(0.01))(x)
x = Dense(64, activation = 'relu')(x)
y = Dense(3, activation = 'softmax')(x)

model = Model(inputs=base_model.input, outputs=y)

In [ ]:
estop = EarlyStopping(monitor='val_loss', patience=4)

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    training,
    validation_data = validation,
    epochs = 25,
    callbacks = [estop]
)

Epoch 1/25
376/376 [==============================] - 5503s 14s/step - loss: 0.9630 - accuracy: 0.5366 - val_loss: 1.5006 - val_accuracy: 0.2518
Epoch 2/25
376/376 [==============================] - 138s 365ms/step - loss: 0.4876 - accuracy: 0.8463 - val_loss: 2.1929 - val_accuracy: 0.3331
Epoch 3/25
376/376 [==============================] - 137s 364ms/step - loss: 0.0856 - accuracy: 0.9931 - val_loss: 1.4542 - val_accuracy: 0.2648
Epoch 4/25
376/376 [==============================] - 137s 364ms/step - loss: 0.4464 - accuracy: 0.8558 - val_loss: 1.6890 - val_accuracy: 0.4437
Epoch 5/25
376/376 [==============================] - 137s 364ms/step - loss: 0.0698 - accuracy: 0.9943 - val_loss: 1.8403 - val_accuracy: 0.3331
Epoch 6/25
376/376 [==============================] - 137s 365ms/step - loss: 0.0660 - accuracy: 0.9953 - val_loss: 2.8002 - val_accuracy: 0.3331
Epoch 7/25
376/376 [==============================] - 138s 365ms/step - loss: 0.5392 - accuracy: 0.7413 - val_loss: 18.0316 -

# Model 2 Evaluation

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

In [ ]:
# classification report
y_pred = model.predict(validation)
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(validation.classes, y_pred))

94/94 [==============================] - 13s 137ms/step
              precision    recall  f1-score   support

           0       0.56      0.19      0.28      1000
           1       0.33      0.88      0.48      1002
           2       0.00      0.00      0.00      1000

    accuracy                           0.36      3002
   macro avg       0.30      0.35      0.25      3002
weighted avg       0.30      0.36      0.25      3002



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Model 3

In [ ]:
base_model = EfficientNetB7(
    input_shape=(X, Y, 3),
    weights = 'imagenet',
    include_top = False)

x = GlobalAveragePooling2D()(base_model.output)
x = Flatten()(x)
x = Dense(128, activation='relu', activity_regularizer=tf.keras.regularizers.L2(0.01))(x)
x = Dense(64, activation = 'relu')(x)
y = Dense(3, activation = 'softmax')(x)

model = Model(inputs=base_model.input, outputs=y)

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    training,
    validation_data = validation,
    epochs = 25
)

Epoch 1/25
376/376 [==============================] - 284s 379ms/step - loss: 1.0108 - accuracy: 0.4843 - val_loss: 1.1462 - val_accuracy: 0.4967
Epoch 2/25
376/376 [==============================] - 137s 365ms/step - loss: 0.6174 - accuracy: 0.7746 - val_loss: 1.6065 - val_accuracy: 0.3085
Epoch 3/25
376/376 [==============================] - 138s 367ms/step - loss: 0.0872 - accuracy: 0.9949 - val_loss: 1.1277 - val_accuracy: 0.3781
Epoch 4/25
376/376 [==============================] - 138s 367ms/step - loss: 0.1249 - accuracy: 0.9828 - val_loss: 1.7547 - val_accuracy: 0.3331
Epoch 5/25
376/376 [==============================] - 137s 365ms/step - loss: 1.1582 - accuracy: 0.3414 - val_loss: 1.1036 - val_accuracy: 0.3131
Epoch 6/25
376/376 [==============================] - 138s 365ms/step - loss: 1.0914 - accuracy: 0.3329 - val_loss: 1.0999 - val_accuracy: 0.3331
Epoch 7/25
 18/376 [>.............................] - ETA: 2:00 - loss: 1.1111 - accuracy: 0.2778

KeyboardInterrupt: ignored

## Model 3 evaluation
- I interrupted this model wbecause there was no evidence of increased validation accuracy and I wanted to save my colab compute units for more promising models. Thus, unfortunately there is no model classification report for this model.

# Model 4

In [ ]:
data_dir = "/content/drive/MyDrive/lung_image_sets"
data = ImageDataGenerator(validation_split=0.2)
BATCH_SIZE = 10

X=224
Y=224

training = data.flow_from_directory(data_dir,
                                    class_mode = 'categorical',
                                    target_size = (X, Y),
                                    color_mode = "rgb",
                                    batch_size = BATCH_SIZE,
                                    shuffle = False,
                                    subset = "training",
                                    seed = 89)

validation = data.flow_from_directory(data_dir,
                                      class_mode = 'categorical',
                                      target_size = (X, Y),
                                      color_mode = "rgb",
                                      batch_size = BATCH_SIZE,
                                      shuffle = False,
                                      subset = 'validation',
                                      seed = 89)

Found 12008 images belonging to 3 classes.
Found 3002 images belonging to 3 classes.


In [ ]:
base_model = EfficientNetB7(
    input_shape=(X, Y, 3),
    weights = 'imagenet',
    include_top = False)

x = GlobalAveragePooling2D()(base_model.output)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation = 'relu', activity_regularizer=tf.keras.regularizers.L2(0.05))(x)
y = Dense(3, activation = 'softmax')(x)

model = Model(inputs=base_model.input, outputs=y)

258076736/258076736 [==============================] - 8s 0us/step


In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
estop = EarlyStopping(monitor='val_loss', patience=4)

In [ ]:
model.fit(
    training,
    validation_data = validation,
    epochs = 25,
    callbacks = [estop]
)

Epoch 1/25
1201/1201 [==============================] - 6908s 6s/step - loss: 1.0199 - accuracy: 0.4453 - val_loss: 1.8129 - val_accuracy: 0.3331
Epoch 2/25
1201/1201 [==============================] - 672s 559ms/step - loss: 0.4120 - accuracy: 0.9176 - val_loss: 2.2372 - val_accuracy: 0.3331
Epoch 3/25
1201/1201 [==============================] - 671s 559ms/step - loss: 0.1482 - accuracy: 0.9961 - val_loss: 2.3013 - val_accuracy: 0.3331
Epoch 4/25
1201/1201 [==============================] - 672s 559ms/step - loss: 0.1889 - accuracy: 0.9534 - val_loss: 3.8465 - val_accuracy: 0.3331
Epoch 5/25
1201/1201 [==============================] - 671s 559ms/step - loss: 0.1565 - accuracy: 0.9868 - val_loss: 1.7560 - val_accuracy: 0.3331
Epoch 6/25
1201/1201 [==============================] - 671s 559ms/step - loss: 0.2087 - accuracy: 0.9460 - val_loss: 2.6717 - val_accuracy: 0.3331
Epoch 7/25
1201/1201 [==============================] - 671s 559ms/step - loss: 0.1376 - accuracy: 0.9838 - val_lo

# Model 5
- Increased regularization weights.

In [4]:
data_dir = "/content/drive/MyDrive/lung_image_sets"
data = ImageDataGenerator(validation_split=0.2)
BATCH_SIZE = 10

X=224
Y=224

training = data.flow_from_directory(data_dir,
                                    class_mode = 'categorical',
                                    target_size = (X, Y),
                                    color_mode = "rgb",
                                    batch_size = BATCH_SIZE,
                                    shuffle = False,
                                    subset = "training",
                                    seed = 89)

validation = data.flow_from_directory(data_dir,
                                      class_mode = 'categorical',
                                      target_size = (X, Y),
                                      color_mode = "rgb",
                                      batch_size = BATCH_SIZE,
                                      shuffle = False,
                                      subset = 'validation',
                                      seed = 89)

Found 12008 images belonging to 3 classes.
Found 3002 images belonging to 3 classes.


In [5]:
base_model = EfficientNetB7(
    input_shape=(X, Y, 3),
    weights = 'imagenet',
    include_top = False)

x = GlobalAveragePooling2D()(base_model.output)
x = Flatten()(x)
x = Dense(128, activation='relu', activity_regularizer=tf.keras.regularizers.L2(0.1))(x)
x = Dense(64, activation = 'relu', activity_regularizer=tf.keras.regularizers.L2(0.1))(x)
y = Dense(3, activation = 'softmax')(x)

model = Model(inputs=base_model.input, outputs=y)

258076736/258076736 [==============================] - 13s 0us/step


In [6]:
estop = EarlyStopping(monitor='val_loss', patience=4)

In [7]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [8]:
model.fit(
    training,
    validation_data = validation,
    epochs = 25,
    callbacks = [estop]
)

Epoch 1/25
1201/1201 [==============================] - 11209s 9s/step - loss: 1.1020 - accuracy: 0.3076 - val_loss: 1.0986 - val_accuracy: 0.3331
Epoch 2/25
1201/1201 [==============================] - 209s 174ms/step - loss: 1.0995 - accuracy: 0.3296 - val_loss: 1.0990 - val_accuracy: 0.3331
Epoch 3/25
1201/1201 [==============================] - 211s 175ms/step - loss: 1.0995 - accuracy: 0.3180 - val_loss: 1.0987 - val_accuracy: 0.3331
Epoch 4/25
1201/1201 [==============================] - 210s 175ms/step - loss: 1.0992 - accuracy: 0.3180 - val_loss: 1.0987 - val_accuracy: 0.3331
Epoch 5/25
1201/1201 [==============================] - 210s 175ms/step - loss: 1.0994 - accuracy: 0.3180 - val_loss: 1.0986 - val_accuracy: 0.3331
Epoch 6/25
1201/1201 [==============================] - 211s 176ms/step - loss: 1.0993 - accuracy: 0.3254 - val_loss: 1.0987 - val_accuracy: 0.3331
Epoch 7/25
1201/1201 [==============================] - 211s 176ms/step - loss: 1.0993 - accuracy: 0.3100 - val_l

In [9]:
# classification report
y_pred = model.predict(validation)
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(validation.classes, y_pred))

301/301 [==============================] - 18s 45ms/step
              precision    recall  f1-score   support

           0       0.33      1.00      0.50      1000
           1       0.00      0.00      0.00      1002
           2       0.00      0.00      0.00      1000

    accuracy                           0.33      3002
   macro avg       0.11      0.33      0.17      3002
weighted avg       0.11      0.33      0.17      3002



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
